In [1]:
import pandas as pd

In [2]:
#This code assums that the name column is the first column
#read csv into dataframes
#one dataframe is worldwide box office data, the other is just the name and year movie was released
df_rev = pd.read_csv('boxofficemojo.csv')
df_years = pd.read_csv('Prophethuge (1).csv', index_col = 0, usecols = ['Name','Date Released'])

#remove rows with unknown values
df_years = df_years[df_years.ne('unknown').all(axis=1)]

#convert date released column to datetime
df_years['year'] = pd.to_datetime(df_years['Date Released']).dt.year

#calculate year differential
for i in df_years:
    df_years['year_diff'] = 2023 - df_years['year']
    

#drop year column as unnecessary, only used to calculate year differential
df_years = df_years.drop(columns='year')

#remove year differential column
df_years = df_years.drop(columns='Date Released')
    
#print dataframe
print(df_years)

                               year_diff
Name                                    
Babylon                                1
A Man Called Otto                      0
Shotgun Wedding                        0
I Wanna Dance with Somebody            1
Narvik: Hitler's First Defeat          0
...                                  ...
Love Liza                             20
Real Women Have Curves                21
The Dancer Upstairs                   21
Tadpole                               21
Biggie and Tupac                      21

[8177 rows x 1 columns]


In [3]:
# Identify columns that are integers
# Remove the second and third columns
df_rev = df_rev.drop(columns=['ID','Distributor'])

# Display the resulting DataFrame
print(df_rev)

                              Name Worldwide Gross Revenue (sus)  \
0         Avatar: The Way of Water                $2,320,250,281   
1     Puss in Boots: The Last Wish                  $481,080,374   
2                          Babylon                   $63,351,455   
3                A Man Called Otto                  $113,190,218   
4                            M3GAN                  $179,973,454   
...                            ...                           ...   
2586              Enter the Dragon                      $114,252   
2587             American Graffiti                  $115,000,889   
2588              Live and Let Die                   $35,382,710   
2589                   Deliverance                        $4,534   
2590                 The Godfather                  $250,341,816   

     Domestic Opening Domestic Gross China Opening   China Gross   UK Opening  \
0        $134,100,226   $684,075,767   $56,798,896  $245,996,272  $13,590,332   
1         $12,429,515

In [4]:
#format all values in all cells
df_rev = df_rev.applymap(lambda x: str(x).replace('$', ''))
df_rev = df_rev.applymap(lambda x: str(x).replace('unknown', '0'))
df_rev = df_rev.applymap(lambda x: str(x).replace(',', ''))

#set the index to the movie name
df_rev.set_index('Name', inplace = True)

#convert entire dataframe to integer to do calculations
df_rev = df_rev.astype('Int64')

#view resulting dataframe
print(df_rev)

                              Worldwide Gross Revenue (sus)  Domestic Opening  \
Name                                                                            
Avatar: The Way of Water                         2320250281         134100226   
Puss in Boots: The Last Wish                      481080374          12429515   
Babylon                                            63351455           3603368   
A Man Called Otto                                 113190218             56257   
M3GAN                                             179973454          30429860   
...                                                     ...               ...   
Enter the Dragon                                     114252                 0   
American Graffiti                                 115000889                 0   
Live and Let Die                                   35382710                 0   
Deliverance                                            4534                 0   
The Godfather               

In [5]:
#merge dataframes on
merged_df = pd.merge(df_rev, df_years, left_index=True, right_index=True)

#define inflation rate
inflation = 0.03       
                           
#multiply all columns in dataframe by the inflation factor (per year)
for col in merged_df.columns:
    if col != 'year_diff':
        merged_df[col] = merged_df[col] * (1 + inflation) ** merged_df['year_diff']
        
#remove year differential column
merged_df = merged_df.drop(columns='year_diff')

#reset index
merged_df.reset_index(inplace=True)

#round all values to 2 decimals
merged_df = merged_df.round(2)

In [6]:
#export dataframe to csv
merged_df.to_csv('inflation_worldwide.csv', index=False)

#view dataframe
merged_df.head()

,Name,Worldwide Gross Revenue (sus),Domestic Opening,Domestic Gross,China Opening,China Gross,UK Opening,UK Gross,France Opening,France Gross,...,Japan Opening,Japan Gross,South Korea Opening,South Korea Gross,Australia Opening,Australia Gross,Russia/CIS Opening,Russia/CIS Gross,India Opening,India Gross
0,(500) Days of Summer,91966126.86,22789221.33,48994859.49,0.0,0.0,0.0,11490927.84,0.0,5573613.31,...,0.0,4518229.54,0.0,2294931.38,0.0,9948886.48,0.0,6518422.23,0.0,446032.46
1,10 Cloverfield Lane,135553005.37,30411628.53,88652995.38,0.0,0.0,2851802.0,8740889.43,0.0,3306972.17,...,0.0,0.0,1646344.82,3106329.32,1352997.77,3755341.14,949094.89,1611151.98,0.0,0.0
2,12 Strong,78087762.63,18333948.47,53117605.37,0.0,0.0,660057.04,1439076.46,436164.12,682984.0,...,933498.49,2270534.49,1156520.8,1992605.49,834294.77,1923770.11,111805.03,172537.08,128432.5,216029.56
3,12 Years a Slave,252297725.11,179947.72,76162419.64,0.0,0.0,0.0,44364223.84,0.0,155160.52,...,0.0,0.0,0.0,186911.89,0.0,0.0,0.0,0.0,0.0,0.0
4,127 Hours,86599001.08,377614.2,26141653.79,0.0,0.0,4808419.94,17841810.63,2131318.62,4127663.31,...,668623.4,668623.4,816675.84,1701545.82,1712249.0,5118987.73,549807.62,1291826.33,758690.14,1392798.72


In [7]:
#do the same for the domestic data

In [19]:
#inflation for the main dataframe
df_g = pd.read_csv('Prophethuge (1).csv', index_col = 0, usecols = ['Name','Estimated Budget','Gross Revenue US&Canada USD', 'Opening Weekend Revenue US&Canada USD', 'Gross Revenue Worldwide USD'])
df_g = df_g.applymap(lambda x: str(x).replace('unknown', '0'))

#convert each column to integer then fill nan values with 0
df_g = df_g.apply(pd.to_numeric, errors='coerce').fillna(0)

#combine with previously made dataframe
df_m = pd.merge(df_g, df_years, left_index=True, right_index=True)

#define constants
current_year = 2023
inflation = 0.03

#multiply each value by inflation factor    
for col in df_m.columns:
    if col != 'year_diff':
        df_m[col] = df_m[col] * (1 + inflation) ** df_m['year_diff']
        
#remove year differential column
df_m = df_m.drop(columns='year_diff')

#round all values to 2 decimals
df_m = df_m.round(2)   

#reset index
df_m.reset_index(inplace=True)

#export to csv
df_m.to_csv('inflation_domestic.csv', index=False)

In [20]:
#print df
df_m

,Name,Estimated Budget,Gross Revenue US&Canada USD,Opening Weekend Revenue US&Canada USD,Gross Revenue Worldwide USD
0,'71,0.000000e+00,1.658167e+06,72755.46,3.995448e+06
1,'83,0.000000e+00,4.302650e+06,1795652.82,7.798741e+06
2,(500) Days of Summer,1.134442e+07,4.899486e+07,1262257.64,9.196613e+07
3,0.45,8.264238e+06,0.000000e+00,0.00,2.831973e+05
4,1,4.031749e+06,1.059812e+04,10598.12,2.251572e+06
...,...,...,...,...,...
8486,iBoy,2.985131e+06,0.000000e+00,0.00,0.000000e+00
8487,xXx,1.302206e+08,2.643653e+08,82794461.82,5.161357e+08
8488,xXx: Return of Xander Cage,1.014944e+08,5.361105e+07,24036442.28,4.132833e+08
8489,xXx: State of the Union,1.923749e+08,4.575107e+07,21641792.14,1.215718e+08
